# Buscadores booleano e bag-of-words no TREC-DL 2020

Aqui, são implementado:
 
*   um buscador booleano, que apenas leva em consideração a ocorrência ou não de cada termo da query em cada documento, independente do número de ocorrências de cada termo.
*   um buscador baseado em bag-of-words, que leva em consideração o número de ocorrências de cada termo no documento.



## Download do dataset

Montagem local para acesso posterior aos arquivos

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
main_path = '/content/drive/MyDrive/Unicamp-aula-2/'

import os

if not os.path.exists(main_path):
  os.makedirs(main_path)
else:
  print('Diretório já existente')

Diretório já existente


## Download de ferramentas auxiliares

Instalação do Pyserini para uso do Lucene Analyzer para fins de pré-processamento(tokenização, remoção de stopwords, stemming).

In [3]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 115.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 115.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 KB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 115.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

Download de ferramentas para avaliação do TREC DL 2020

In [ ]:
!git clone https://github.com/castorini/pyserini.git --recurse-submodules {main_path}/pyserini

fatal: destination path '/content/drive/MyDrive/Unicamp-aula-2//pyserini' already exists and is not an empty directory.


In [ ]:
!cd {main_path}/pyserini/tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
!cd {main_path}/pyserini/tools/eval/ndeval && make && cd ../../..

trec_eval.9.0.4/
trec_eval.9.0.4/m_prefs_pair.c
trec_eval.9.0.4/m_ndcg_p.c
trec_eval.9.0.4/m_infap.c
trec_eval.9.0.4/m_num_q.c
trec_eval.9.0.4/m_iprec_at_recall.c
trec_eval.9.0.4/form_prefs_counts.c
trec_eval.9.0.4/m_prefs_num_prefs_ful_ret.c
trec_eval.9.0.4/utility_pool.c
trec_eval.9.0.4/m_binG.c
trec_eval.9.0.4/meas_avg.c
trec_eval.9.0.4/m_gm_bpref.c
trec_eval.9.0.4/m_runid.c
trec_eval.9.0.4/m_bpref.c
trec_eval.9.0.4/m_gm_map.c
trec_eval.9.0.4/trec_eval.h
trec_eval.9.0.4/m_yaap.c
trec_eval.9.0.4/m_relstring.c
trec_eval.9.0.4/m_Rprec.c
trec_eval.9.0.4/m_prefs_avgjg.c
trec_eval.9.0.4/m_success.c
trec_eval.9.0.4/m_ndcg.c
trec_eval.9.0.4/functions.h
trec_eval.9.0.4/m_P_avgjg.c
trec_eval.9.0.4/test/
trec_eval.9.0.4/test/qrels.rel_level
trec_eval.9.0.4/test/results.test
trec_eval.9.0.4/test/qrels.test
trec_eval.9.0.4/test/out.test.qrels_jg
trec_eval.9.0.4/test/out.test.meas_params
trec_eval.9.0.4/test/out.test.a
trec_eval.9.0.4/test/out.test.prefs
trec_eval.9.0.4/test/out.test.aqcM
trec_ev

## Construção dos índice invertidos.

Inicialmente foi construído um índice invertido apenas para a busca booleana, que foi utilizado em seguida para os tetes.  

Posteriormente, foi acrescentado um índice invertido para buscas do tipo bag-of-words, onde a única diferença em relação à busca booleana é que é levada em consideração o número de vezes em que cada termo ocorre em cada documento, enquanto na booleana apenas a presença de cada termo da busca é checada.

In [4]:
from pyserini.analysis import Analyzer, get_lucene_analyzer

In [5]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


Código para pré-processamento textual

In [6]:
analyzer = Analyzer(get_lucene_analyzer())

def preprocess_and_tokenize(text):
  return analyzer.analyze(text)

In [7]:
collection_path = main_path + '/collections/msmarco-passage/collection.tsv'

Foi verificado que o analisador do Lucene não elimina suficientemente stopwords.  Assim, foi utilizada adicionalmente a lista de stopwords do NLTK.

In [8]:
import nltk
import string

from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

nltk.download('stopwords')  # Download stopwords if not already downloaded

from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words = set(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Em seguida, será construído o índice invertido para associar os tokens aos documentos nos quais eles ocorrem.  O uso da estrutura de dados array (sugestão do colega [Leandro Carísio](https://colab.research.google.com/drive/1hELJYqsvUyja9HPeDzc9FU8okqdIjODE?usp=sharing)) resultou em um ganho relevante em memória em relação a listas.  Tal estrutura de dados, ao contrário dos arrays do Numpy, assemelha-se a um array dinâmico, ou lista, porém ocupa bem menos memória.

Índice invertido para busca booleana:

In [9]:

import pandas as pd
from collections import defaultdict
import array
import pickle

index_path = f"{main_path}/inverted_index_boolean.pickle"

if os.path.exists(index_path):
    with open(index_path, "rb") as f:
      print("Loading index...")
      inverted_index_boolean = pickle.load(f)
else:
  # set the chunk size
  chunk_size = 1000
  chunks = []
  inverted_index_boolean = dict()
  full_text = ''

  def process(row):
    tokenized_text = preprocess_and_tokenize(row[1])
    doc_id = row[0]
    for token in tokenized_text:
      if token not in stop_words:
        inverted_index_boolean.setdefault(token, array.array("L", [])).append(int(doc_id))

  chunk_id = 0
  # iterate through the file in chunks
  for chunk in pd.read_csv(collection_path, sep='\t', header=None, chunksize=chunk_size):
    # process the chunk here
    if (chunk_id % 1000) == 0:
      print(f'Processing chunk {chunk_id}')
    for index, row in chunk.iterrows():
      process(row)
    del(chunk)
    chunk_id += 1

  with open(index_path, "wb") as f:
    pickle.dump(inverted_index_boolean, f)

Loading index...


Índice invertido para busca baseada em bag-of-words:

In [10]:
import pandas as pd
from collections import defaultdict
import array
import pickle
from collections import Counter

index_path = f"{main_path}/inverted_index_bow.pickle"

if os.path.exists(index_path):
    with open(index_path, "rb") as f:
      print("Loading index...")
      inverted_index_bow = pickle.load(f)
else:
  # set the chunk size
  chunk_size = 1000
  chunks = []
  inverted_index_bow = dict()
  full_text = ''

  def process(row):
    tokenized_text = preprocess_and_tokenize(row[1])
    counter = Counter(tokenized_text)
    doc_id = row[0]
    for token, count in counter.items():
      if token not in stop_words:
        inverted_index_bow.setdefault(token, {"docs":array.array("L", []), "counts":array.array("f", [])})["docs"].append(int(doc_id))
        inverted_index_bow.setdefault(token, {"docs":array.array("L", []), "counts":array.array("L", [])})["counts"].append(count)

  chunk_id = 0
  # iterate through the file in chunks
  for chunk in pd.read_csv(collection_path, sep='\t', header=None, chunksize=chunk_size):
    # process the chunk here
    if (chunk_id % 1000) == 0:
      print(f'Processing chunk {chunk_id}')
    for index, row in chunk.iterrows():
      process(row)
    del(chunk)
    chunk_id += 1

  with open(index_path, "wb") as f:
    pickle.dump(inverted_index_bow, f)

Loading index...


In [ ]:
len(inverted_index_boolean)

2660662

In [ ]:
assert len(inverted_index_boolean) == len(inverted_index_bow)

In [11]:
!head {collection_path}

0	The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.
1	The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science.
2	Essay on The Manhattan Project - The Manhattan Project The Manhattan Project was to see if making an atomic bomb possible. The success of this project would forever change the world forever making it known that something this powerful can be manmade.
3	The Manhattan Project was the name for a project conducted during World War II, to develop the first atomic bomb. It refers specifically to the period of the project from 194 â¦ 2-1946 under the control of the U.S. Army Corps of Engineers

## Avaliação

In [12]:
topics_file = main_path + '/pyserini/tools/topics-and-qrels/topics.dl20.txt'
qrels_eval = main_path + '/pyserini/tools/topics-and-qrels/qrels.dl20-passage.txt'

In [13]:
!head {topics_file}

1030303	who is aziz hashim
1037496	who is rep scalise?
1043135	who killed nicholas ii of russia
1045109	who owns barnhart crane
1049519	who said no one can make you feel inferior
1051399	who sings monk theme song
1056416	who was the highest career passer  rating in the nfl
1064670	why do hunters pattern their shotguns?
1065636	why do some places on my scalp feel sore
1071750	why is pete rose banned from hall of fame


In [14]:
!head {qrels_eval}

23849 0 1020327 2
23849 0 1034183 3
23849 0 1120730 0
23849 0 1139571 1
23849 0 1143724 0
23849 0 1147202 0
23849 0 1150311 0
23849 0 1158886 2
23849 0 1175024 1
23849 0 1201385 0


Lógica de busca booleana

In [15]:
def search_boolean(query, use_stopwords=False):
  doc_scores = defaultdict(int) # int (doc_id) -> int (score)
  query_tokens = preprocess_and_tokenize(query)
  query_counter = Counter(query_tokens)

  for token in query_counter.keys():
    if token in inverted_index_boolean:
      doc_ids = set(inverted_index_boolean[token])
      for doc_id in doc_ids:
        doc_scores[doc_id] += 1

  return doc_scores

Lógica de busca baseada em bag-of-words

In [16]:
import math

def search_bow(query):
  doc_scores = defaultdict(int) # int (doc_id) -> int (score)
  query_tokens = preprocess_and_tokenize(query)
  n_query_tokens = len(query_tokens)
  query_counter = Counter(query_tokens)

  for token in query_counter.keys():
    if token in inverted_index_bow:
      doc_ids = inverted_index_bow[token]["docs"]

      for i, doc_id in enumerate(doc_ids):
        doc_scores[doc_id] += inverted_index_bow[token]["counts"][i]
    
          
  return doc_scores

In [17]:
results_boolean = search_boolean('who is aziz hashim', True)
results_bow = search_bow('who is aziz hashim')

In [ ]:
print(len(results_boolean))
print(len(results_bow))

245
245


In [18]:
def get_document_by_id(id):
  result = None
  with open(collection_path, 'r') as f:
    for line in f:
      fields = line.strip().split('\t')
      doc_id = fields[0]
      if doc_id == id:
        result = fields[1]
        break

  return result

In [19]:
sorted_results_boolean = sorted(results_boolean.items(), key=lambda x: x[1], reverse=True)[:10]
sorted_results_bow = sorted(results_bow.items(), key=lambda x: x[1], reverse=True)[:10]

In [20]:
sorted_results_boolean

[(7156982, 2),
 (8726429, 2),
 (8726430, 2),
 (8726433, 2),
 (8726434, 2),
 (8726435, 2),
 (8726436, 2),
 (8726437, 2),
 (794624, 1),
 (4820481, 1)]

In [21]:
sorted_results_bow

[(8726436, 8.0),
 (1305520, 6.0),
 (1305521, 6.0),
 (6222298, 5.0),
 (1451846, 4.0),
 (1905988, 3.0),
 (2699097, 3.0),
 (6764572, 3.0),
 (6939422, 3.0),
 (6939426, 3.0)]

In [ ]:
get_document_by_id('6939426')

"Abdul Aziz (Arabic: Ø¹Ø¨Ø¯ Ø§Ù\x84Ø¹Ø²Ù\x8aØ² â\x80\x8e) is a male Muslim given name and in modern usage, surname. It is built from the Arabic words Abd, al-and Aziz.The name means servant of the Almighty, Al-AzÄ«z being one of the names of God in the Qur'an, which give rise to the Muslim theophoric names.bdul Aziz (Arabic: Ø¹Ø¨Ø¯ Ø§Ù\x84Ø¹Ø²Ù\x8aØ² â\x80\x8e) is a male Muslim given name and in modern usage, surname. It is built from the Arabic words Abd, al-and Aziz."

As próximas células executam as rotinas de preparação e cálculo das métricas segundo o dataset de teste do TREC DL 2020.

In [22]:
!python3 {main_path}/pyserini/tools/scripts/msmarco/filter_queries.py \
--qrels <(sed -e 's/ /\t/g' {qrels_eval}) \
--queries {topics_file} \
--output topics.dl20.small.tsv

Done!


In [23]:
topics_file = 'topics.dl20.small.tsv'

In [24]:
query_to_results_boolean = dict()
query_to_results_bow = dict()

with open(topics_file, 'r') as f:
  for line in f:
      fields = line.strip().split('\t')
      query_id = fields[0]
      query_text = fields[1]
      results_boolean = search_boolean(query_text)
      query_to_results_boolean[int(query_id)] = sorted(results_boolean.items(), key=lambda x: x[1], reverse=True)[:10]
      results_bow = search_bow(query_text)
      query_to_results_bow[int(query_id)] = sorted(results_bow.items(), key=lambda x: x[1], reverse=True)[:10]

with open('run.dl20.boolean.trec', 'w') as f:
  for query_id, results in query_to_results_boolean.items():
    for i, (doc_id, score) in enumerate(results):
      f.write(f'{query_id}\tQ0\t{doc_id}\t{i+1}\t{score}\tboolean\n')

with open('run.dl20.bow.trec', 'w') as f:
  for query_id, results in query_to_results_bow.items():
    for i, (doc_id, score) in enumerate(results):
      f.write(f'{query_id}\tQ0\t{doc_id}\t{i+1}\t{score}\tbow\n')
  

In [25]:
!head run.dl20.boolean.trec

1030303	Q0	7156982	1	2	boolean
1030303	Q0	8726429	2	2	boolean
1030303	Q0	8726430	3	2	boolean
1030303	Q0	8726433	4	2	boolean
1030303	Q0	8726434	5	2	boolean
1030303	Q0	8726435	6	2	boolean
1030303	Q0	8726436	7	2	boolean
1030303	Q0	8726437	8	2	boolean
1030303	Q0	794624	9	1	boolean
1030303	Q0	4820481	10	1	boolean


In [26]:
!python {main_path}/pyserini/tools/scripts/msmarco/convert_msmarco_to_trec_qrels.py \
   --input {qrels_eval} \
   --output qrels.dl20.trec

Done!


In [27]:
!head qrels.dl20.trec

23849 0 1020327 2
23849 0 1034183 3
23849 0 1120730 0
23849 0 1139571 1
23849 0 1143724 0
23849 0 1147202 0
23849 0 1150311 0
23849 0 1158886 2
23849 0 1175024 1
23849 0 1201385 0


In [28]:
!chmod 755 {main_path}/pyserini/tools/eval/trec_eval.9.0.4/trec_eval

Métricas para busca booleana

In [29]:
!{main_path}/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   qrels.dl20.trec run.dl20.boolean.trec

map                   	all	0.1117
ndcg_cut_10           	all	0.3189


Métricas para busca baseada em bag-of-words

In [30]:
!{main_path}/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   qrels.dl20.trec run.dl20.bow.trec

map                   	all	0.0153
ndcg_cut_10           	all	0.0492
